# Precheck

This book gives you an opportunity to check your strategy before submitting it. 
It is developed in a way which can catch most errors in your strategies.
Also, it checks general filters for submissions.
Pay attention to the red outputs.

In [ ]:
from qnt.precheck import *

# set the competition type here:
data_type = 'stocks' # 'stocks', 'futures', 'crypto'

The 1st step runs `init.ipynb` in the test environment in order to install additional dependencies.

In [ ]:
run_init()

If you want to reset your environment in order to properly test the installation of external dependencies,
you can run the command below. 
This command will shutdown the jupyter instance and remove your current environment. 
After that, you have to reopen this jupyter instance from your personal space.
The system will start a new instance a create a new conda environment.
You won't loose your code. It will be deployed in the new instance.

In [ ]:
# ! kill -9 -1

The 2nd step runs your strategy multiple times cutting the data tail. It is similar to the step-by-step calculation.

By default, it runs 3 passes. It is enough to catch most errors. If you want to get more relevant results, you need to increase the number of passes. When you submit your strategy, the system runs about 1000 passes, which requires a very long time.

Also, this step performs some intermediate checks of the outputs.

When passes finish, you can find results in the folder `precheck_results`.

In [ ]:
evaluate_passes(passes=3, data_type=data_type)

# you can also set the the specific dates:
# evaluate_passes(dates=['2016-09-30', '2018-07-13', '2020-04-23'])

The 3rd step assembles the output from the outputs calculated in the previous step.

In [ ]:
output = assemble_output()

And the final step checks the major filters: points count, sharpe ratio and correlation.

In [ ]:
check_output(output, data_type=data_type)